In [16]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [17]:
# Función para extraer los datos de una URL
def extraer_datos_url(url):
    # Configuración del navegador
    browser.get(url)
    sleep(2)  # Esperar que cargue la página
    
    # Extraer nombre (título)
    try:
        nombre = browser.find_element(By.XPATH, "//h1").text
    except:
        nombre = None

    # Extraer precio
    try:
        precio = browser.find_element(By.CSS_SELECTOR, ".price__value.jsPriceValue").text
    except:
        precio = None

    return {"URL": url, "Nombre": nombre, "Precio": precio}

# Configuración del navegador
browser = webdriver.Chrome()
browser.get("https://www.pisos.com/")
browser.maximize_window()

# Aceptar cookies si es necesario
browser.find_element(by=By.CSS_SELECTOR, value="#didomi-notice-disagree-button").click()
sleep(1)

# Buscar "Madrid"
buscador = browser.find_element(by=By.XPATH, value='/html/body/main/div[1]/div/div[2]/div/div[2]/div/div[1]/input')
buscador.click()
sleep(1)
buscador.send_keys("Madrid")
sleep(1)

# Hacer clic en "Provincia"
browser.find_element(by=By.XPATH, value="//div[@class='location-item__subtitle' and text()='Provincia']").click()
sleep(1)

# Almacenar la URL actual como provincia_madrid
provincia_madrid = browser.current_url
print(provincia_madrid)  # Imprime el URL actual

# Inicializar la lista para almacenar los resultados
datos = []

# Acceder a las diferentes zonas y extraer las URLs
browser.find_element(by=By.CSS_SELECTOR, value="#main > div.inner-body.with-footer > div.content.subHome.subhome__wrapper > div:nth-child(5) > div.subhome__content > div:nth-child(1) > div.zoneList > nav > div > ul > li:nth-child(1) > a.seo-box__location-link.seo-box__location-link--level2").click()
sleep(1)

# Comarca1
comarca1_url = browser.current_url
print(f"Comarca1:{comarca1_url} ")

# Municipio1 Comarca1
browser.find_element(by=By.CSS_SELECTOR, value="#main > div.inner-body.with-footer > div.content.subHome.subhome__wrapper > div:nth-child(5) > div.subhome__content > div:nth-child(1) > div.zoneList > nav > div > ul > li:nth-child(1) > a.seo-box__location-link.seo-box__location-link--level2").click()
sleep(1)

municipio1_comarca1_url = browser.current_url
print(f"Municipio1 Comarca1:{municipio1_comarca1_url} ")

# Recoger la URL de los anuncios
browser.find_element(by=By.XPATH, value="//*[@id=\"5244455514.109700\"]").click()
sleep(1)

# Extraer datos de la URL actual
datos.append(extraer_datos_url(browser.current_url))

# Volver atrás y seguir con el siguiente
browser.back()
browser.find_element(by=By.XPATH, value="//*[@id=\"5227104703.109700\"]").click()
sleep(1)

# Extraer datos de la siguiente URL
datos.append(extraer_datos_url(browser.current_url))

# Convertir los resultados en un DataFrame de Pandas
df = pd.DataFrame(datos)

# Mostrar el DataFrame
print(df)

# Cerrar el navegador
browser.quit()

https://www.pisos.com/viviendas/madrid/
Comarca1:https://www.pisos.com/viviendas/corredor_del_henares/ 
Municipio1 Comarca1:https://www.pisos.com/venta/pisos-ajalvir/ 
                                                 URL  \
0  https://www.pisos.com/comprar/atico-ajalvir_ce...   
1  https://www.pisos.com/comprar/atico-ajalvir_ce...   

                              Nombre     Precio  
0  Ático en venta en Calle Real, s/n  329.700 €  
1  Ático en venta en Calle Real, s/n  177.700 €  


In [18]:
df.head()

,URL,Nombre,Precio
0,https://www.pisos.com/comprar/atico-ajalvir_ce...,"Ático en venta en Calle Real, s/n",329.700 €
1,https://www.pisos.com/comprar/atico-ajalvir_ce...,"Ático en venta en Calle Real, s/n",177.700 €


In [ ]:
# URL de la provincia de Madrid
provincia_url = "https://www.pisos.com/viviendas/madrid/"

# Configuración del navegador
browser = webdriver.Chrome()
browser.get(provincia_url)  # Abrimos la página de Madrid directamente
browser.maximize_window()

# Aceptar cookies si es necesario
browser.find_element(by=By.CSS_SELECTOR, value="#didomi-notice-disagree-button").click()
sleep(1)

# Función para obtener las comarcas
def obtener_comarcas():
    return browser.find_elements(by=By.CSS_SELECTOR, value="a.seo-box__location-link--level2")

# Función para extraer el número de resultados del botón
def obtener_numero_resultados(boton):
    try:
        # Extraer el texto del botón
        texto_boton = boton.text
        # Buscar el número de resultados en el texto
        numero_resultados = int(''.join(filter(str.isdigit, texto_boton)))  # Extrae los dígitos del texto
        return numero_resultados
    except Exception as e:
        print(f"No se pudo extraer el número de resultados: {e}")
        return 0  # Si no se puede extraer, consideramos que no hay resultados

# Primero, obtener todas las comarcas de la página de Madrid
comarcas = obtener_comarcas()

# Crear una lista con las URLs de las comarcas
comarcas_urls = []
for comarca in comarcas:
    comarca_url = comarca.get_attribute("href")
    if "viviendas" in comarca_url and not ("venta" in comarca_url or "alquiler" in comarca_url):
        comarcas_urls.append(comarca_url)

# Iterar sobre todas las URLs de las comarcas
for comarca_url in comarcas_urls:
    print(f"Accediendo a la comarca: {comarca_url}")

    # Acceder a la página de la comarca
    browser.get(comarca_url)
    sleep(2)

    # Intentar encontrar el botón
    try:
        boton = browser.find_element(by=By.CSS_SELECTOR, value="#main > div.inner-body.with-footer > div.content.subHome.subhome__wrapper > div:nth-child(5) > div:nth-child(1) > h1 > a")
        
        # Verificar el número de resultados
        numero_resultados = obtener_numero_resultados(boton)
        print(f"Número de resultados encontrados: {numero_resultados}")

        # Solo hacer clic si el número de resultados es menor o igual a 3100
        if numero_resultados <= 3100:
            boton.click()
            print("Haciendo clic en el botón...")
            sleep(2)
        else:
            print("Número de resultados mayor a 3100, no se hace clic.")

    except Exception as e:
        print(f"No se encontró el botón o ocurrió un error: {e}")

    # Volver a la página de la provincia de Madrid
    browser.get(provincia_url)  # Ir directamente a la página de la provincia
    sleep(2)

    # Volver a obtener las comarcas después de recargar la página de la provincia
    sleep(3)  # Asegúrate de dar tiempo a que la página cargue completamente
    comarcas = obtener_comarcas()  # Ahora obtiene las comarcas nuevamente

# Cerrar el navegador al terminar
browser.quit()

In [19]:
import pandas as pd

In [27]:
df = pd.read_csv('cps.csv', sep=';')

In [28]:
df_provincia = df['provincia']

In [37]:
provincias_unicas = df['provincia'].unique()

In [38]:
provincias_unicas

array(['Araba/Álava', 'Albacete', 'Alicante/Alacant', 'Almería', 'Ávila',
       'Badajoz', 'Illes Balears', 'Barcelona', 'Burgos', 'Cáceres',
       'Cádiz', 'Castellón/Castelló', 'Ciudad Real', 'Córdoba',
       'A Coruña', 'Cuenca', 'Girona', 'Granada', 'Guadalajara',
       'Gipuzkoa', 'Huelva', 'Huesca', 'Jaén', 'León', 'Lleida',
       'La Rioja', 'Lugo', 'Madrid', 'Málaga', 'Murcia', 'Navarra',
       'Ourense', 'Asturias', 'Palencia', 'Las Palmas', 'Pontevedra',
       'Salamanca', 'Santa Cruz de Tenerife', 'Cantabria', 'Segovia',
       'Sevilla', 'Soria', 'Tarragona', 'Teruel', 'Toledo',
       'Valencia/València', 'Valladolid', 'Bizkaia', 'Zamora', 'Zaragoza',
       'Ceuta', 'Melilla'], dtype=object)

In [41]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep

In [42]:
# Función para extraer los datos de una URL
def extraer_datos_url(url):
    browser.get(url)
    sleep(2)  # Esperar que cargue la página
    
    try:
        nombre = browser.find_element(By.XPATH, "//h1").text
    except:
        nombre = None

    try:
        precio = browser.find_element(By.CSS_SELECTOR, ".price__value.jsPriceValue").text
    except:
        precio = None

    return {"URL": url, "Nombre": nombre, "Precio": precio}

# Función para procesar una zona específica
def procesar_zona(url):
    browser.get(url)
    sleep(2)
    
    try:
        resultados = int(browser.find_element(By.CSS_SELECTOR, ".list-title__count").text.replace('.', ''))
    except:
        resultados = 0
    
    if resultados < 3000:
        datos.append(extraer_datos_url(url))
    else:
        enlaces = browser.find_elements(By.CSS_SELECTOR, "a.seo-box__location-link.seo-box__location-link--level2")
        for enlace in enlaces:
            procesar_zona(enlace.get_attribute("href"))

# Configuración del navegador
browser = webdriver.Chrome()
browser.get("https://www.pisos.com/")
browser.maximize_window()
sleep(2)

# Aceptar cookies si es necesario
try:
    browser.find_element(By.CSS_SELECTOR, "#didomi-notice-disagree-button").click()
    sleep(1)
except:
    pass

# Lista de provincias a buscar
provincias = ["Madrid", "Barcelona", "Valencia"]

# Inicializar la lista para almacenar los resultados
datos = []

for provincia in provincias:
    buscador = browser.find_element(By.XPATH, '/html/body/main/div[1]/div/div[2]/div/div[2]/div/div[1]/input')
    buscador.clear()
    buscador.send_keys(provincia)
    sleep(1)
    
    browser.find_element(By.XPATH, "//div[@class='location-item__subtitle' and text()='Provincia']").click()
    sleep(2)
    
    procesar_zona(browser.current_url)

# Convertir los resultados en un DataFrame de Pandas
df = pd.DataFrame(datos)

# Mostrar el DataFrame
print(df)

# Cerrar el navegador
browser.quit()

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x00000001012202d4 cxxbridge1$str$ptr + 2739836
1   chromedriver                        0x0000000101218934 cxxbridge1$str$ptr + 2708700
2   chromedriver                        0x0000000100d79f90 cxxbridge1$string$len + 93360
3   chromedriver                        0x0000000100dc0de4 cxxbridge1$string$len + 383748
4   chromedriver                        0x0000000100e01e80 cxxbridge1$string$len + 650144
5   chromedriver                        0x0000000100db5060 cxxbridge1$string$len + 335232
6   chromedriver                        0x00000001011e8c38 cxxbridge1$str$ptr + 2512864
7   chromedriver                        0x00000001011ebf58 cxxbridge1$str$ptr + 2525952
8   chromedriver                        0x00000001011ce578 cxxbridge1$str$ptr + 2404640
9   chromedriver                        0x00000001011ec818 cxxbridge1$str$ptr + 2528192
10  chromedriver                        0x00000001011bef2c cxxbridge1$str